### 一、拉数据

In [1]:
import pymysql
import pandas as pd
import numpy as np

In [21]:
conn=pymysql.connect(host='mysql-backup.shanzhen.me',
                     user='readonly_medicine_liuqq',
                     password="r3g1^QWgV8GeYJw4",
                     database='asgard_dataplatform',
                     port= 3001,
                     charset='utf8')

In [22]:
def get_SqlData(path):
    dict_ = pd.read_excel(path)
    id_list = dict_.loc[:,'TagID'].values.tolist()
    tag_df = pd.DataFrame(columns=['ORDER_CODE', 'TAG_ID'])
    for items in id_list:
        id_str = "'"+ items + "'"
        tagsql = 'SELECT p1.ORDER_CODE,p1.TAG_ID,p2.EXAM_DATE,p2.EXAMINEE_AGE,p2.EXAMINEE_GENDER FROM (SELECT * FROM T66_APOLLO_ANALYSIS_TAG WHERE SOURCE_TYPE = 2 AND SZ_STATUS = 1 AND TAG_ID IN ('+id_str+') )p1 LEFT JOIN (SELECT * FROM t66_athena_basic_raw_person_info WHERE SZ_STATUS = 1)p2 \
             ON p1.ORDER_CODE = p2.ORDER_CODE' 
        tag = pd.read_sql(tagsql,con=conn)
        tag_df = pd.concat([tag_df,tag],axis = 0)
    return tag_df

FL_tag_df = get_SqlData(r"C:\Users\13670\Desktop\待完成\006 时间序列模型\疾病检出率预测\TS_脂肪肝.xls")

KeyboardInterrupt: 

In [18]:
FL_tag_df

,ORDER_CODE,TAG_ID,EXAM_DATE,EXAMINEE_GENDER
0,000014659233,OBDIM0002Tag0013,2019-10-18 00:00:00,MW
1,000237571502,OBDIM0002Tag0013,2019-03-05 00:00:00,M
2,000516209438,OBDIM0002Tag0013,2018-12-20 00:00:00,M
3,000534638741,OBDIM0002Tag0013,2018-11-05 00:00:00,M
4,000604466170,OBDIM0002Tag0013,2018-07-27 09:00:00,MW
...,...,...,...,...
2251,8kh45xy4zl2q5pf,OBDIM0002Tag0031,NaT,None
2252,8kh4698fvx8kgrr,OBDIM0002Tag0031,NaT,None
2253,8kh46ah7iiz88sm,OBDIM0002Tag0031,NaT,None
2254,8kh46i9gaoysqdp,OBDIM0002Tag0031,NaT,None


In [19]:
FL_tag_df.to_pickle(r"C:\Users\13670\Desktop\待完成\006 时间序列模型\疾病检出率预测\脂肪肝.pkl")

In [ ]:
FL_tagall = FL_tag_df.copy()
FLdict = pd.read_excel(r"C:\Users\13670\Desktop\待完成\006 时间序列模型\疾病检出率预测\TS_脂肪肝.xls")
dict_abbr = FLdict.set_index("TagID").to_dict()["abbr1"]
FL_tagall.replace(dict_abbr,inplace=True)
tagall_abbr1 = FL_tagall.drop_duplicates()

### 二、时间序列预测

In [ ]:
def obtain_DatetimeData():
    